In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# dnn

In [ ]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_circles
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from pandas import read_csv
from keras.optimizers import SGD
import numpy as np
import pandas as pd
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load  dataset
data = pd.read_csv('../input/autism-screening-child-two-version/Child-Data2017.csv', na_values='?')
data.replace("?",np.nan,inplace=True) #replace ? with nan
data.age[62]=data.age.median()
data.age[91]=data.age.median()
# format all fields as string
for column in data.columns:
    if data[column].dtype == type(object):
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))

dataset = data.values
X = dataset[:,0:21]
Y = dataset[:,21]
# reshape target to be a 2d array
Y = Y.reshape((len(Y), 1))
#X, Y = make_circles(n_samples=1000, noise=0.1, random_state=1) #Accuracy: 49.09
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
model = Sequential()
model.add(Dense(1, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
#model.add(Dense(5,  activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
epochs = 50
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
#history =model.fit(X_train, y_train, epochs=100,verbose=2 )
history =model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=100,  batch_size=len(X_train))
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))  

cpu Accuracy: 52.58
gpu Accuracy: 52.58
tpu Accuracy: 52.58




In [ ]:
from matplotlib import pyplot
# plot loss learning curves
'''pyplot.subplot(211)
pyplot.title('Cross-Entropy Loss', pad=-200)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()'''
# plot accuracy learning curves
pyplot.subplot(212)
pyplot.title('Accuracy', pad=-500)
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'test'])
    
plt.show() 

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# cnn

In [ ]:
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import pandas as pd
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
data = pd.read_csv('../input/autism-screening-child-two-version/Child-Data2017.csv', na_values='?')
# format all fields as string
for column in data.columns:
    if data[column].dtype == type(object):
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))

dataset = data.values
X = dataset[:,0:21]
Y = dataset[:,21]
# reshape target to be a 2d array
Y = Y.reshape((len(Y), 1))
X = X.reshape(X.shape[0], X.shape[1], 1)
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(21,1)))
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'binary_crossentropy', 
     optimizer = "adam",               
              metrics = ['accuracy'])
#model.summary()
#acc = model.evaluate(X_train, y_train)
#print('acc:',acc,"Loss:", acc[0], " Accuracy:", acc[1])
history =model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  batch_size=len(X_train))
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))      
pred = model.predict(X_test)
pred_y = pred.argmax(axis=-1)
cm = confusion_matrix(y_test, pred_y)
print('confusion_matrix\n',cm)

Accuracy: 52.58


In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(history.history.keys())